In [1]:
#!pip install xmltodict

In [1]:
import xmltodict
import json
import pandas as pd

In [2]:
periods = [2017, 2018, 2019] # 1980, 1982, 1985, 1987, 1989, 1990, 1992, 1995, 2000, 2001, 2002, 2003, 2005, 2008, 2009, 2010, 2012, 2013, 2015, 2016
all_data = {}

In [3]:
for period in periods:
    period = str(period)

    with open('../articles_raw_data/NewYorkTimes.' + period +'.xml') as xml_file:
        all_data[period] = xmltodict.parse(xml_file.read())

In [4]:
for period in periods:
    period = str(period)

    a = len(all_data[period]['records']['record'])
    data = {}
    c = 0

    for i in range(a):

        if 'fulltext' not in all_data[period]['records']['record'][i].keys():
            continue

        elif c < 20000:
            data[i] = all_data[period]['records']['record'][i]
            c += 1
        else:
            break

    all_data[period] = pd.DataFrame.from_dict(data, orient='index')


In [5]:
for period in periods:
    period = str(period)
    all_data[period] = all_data[period].fillna('')

In [6]:
for period in periods:
    period = str(period)
    all_data[period]['year'] = list(map(lambda x: int(x[:4]), all_data[period]['publishDate']))

    all_data[period]['text'] =  all_data[period]['fulltext'] #all_data[period]['leadtext'] +

    all_data[period].drop(['record_id', 'source_id', 'publishDate', 'series', 'series2', 'author', 'title', 'link', 'leadtext', 'fulltext'], axis = 1, inplace= True)

In [7]:
for period in periods:
    period = str(period)
    all_data[period] = all_data[period][all_data[period]['year'] == int(period)]

In [8]:
for period in periods:
    period = str(period)

    all_data[period]['word_count'] = list(map(lambda x: len(str(x).split()), all_data[period]['text']))

    all_data[period] = all_data[period][all_data[period]['word_count'] > 20]

# Pre-processing 

In [9]:
import re
import numpy as np
import nltk

In [12]:
def preprocess(text_string:str):
    
    non_word_char = "[^\w\'\.]"
    quotations = "\'\'"

    text_string = ' '.join(text_string)

    parsed_text = text_string.lower().split()

    parsed_text = list(map(lambda x: x[:-1] if (x[-1] == '.') and (len(x) < 4) else x, parsed_text))
    parsed_text = list(map(lambda x: 'mrs' if x == 'mrs.' else x, parsed_text))

    parsed_text = ' '.join(parsed_text)

    parsed_text = re.sub(non_word_char, ' ', parsed_text)
    parsed_text = re.sub(quotations, '', parsed_text)
    
    return parsed_text

In [13]:
for period in periods:
    period = str(period)


    print(all_data[period].shape)

(17240, 3)
(17860, 3)
(17527, 3)


In [14]:
for p in periods:
    print(p)

    all_data[str(p)]['text'] = np.vectorize(preprocess)(all_data[str(p)]['text'])
    all_data[str(p)].to_csv('../articles_raw_data/news_' + str(p) + '.csv', index = False)

2017
2018
2019
